In [20]:
import os
import pyodbc
import langchain
from langchain_groq import ChatGroq
from langchain_community.llms import HuggingFaceHub
from langchain_huggingface import HuggingFaceEndpoint
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI

load_dotenv(find_dotenv())

api_key=os.environ["HF_TOKEN"]

repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1"
llm=HuggingFaceEndpoint(repo_id=repo_id, temperature=0.2)
repo_id_deepSeek="deepseek-ai/DeepSeek-R1-Distill-Qwen-32B"
llm_deepseek=HuggingFaceEndpoint(repo_id=repo_id_deepSeek, temperature=0.4)
repo_id_meta="meta-llama/Llama-3.1-8B-Instruct"
#llm_meta=HuggingFaceEndpoint(repo_id=repo_id_meta, temperature=0.1)
#llm_groq=ChatGroq(model='llama-3.3-70b-versatile', api_key=os.environ["GROQ"])
llm_groq=ChatGroq(model='gemma2-9b-it', api_key=os.environ["GROQ"])
llm_openai = ChatOpenAI(model="gpt-4o-mini", temperature=0.4)


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


**DB Setup**

In [21]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
from langchain_community.utilities.sql_database import SQLDatabase

server = 'ASUS\\SQLEXPRESS' 
database = 'AAD' 

# Create a connection string for SQLAlchemy using Windows Authentication
connection_string = f'mssql+pyodbc://@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes'
# Create a connection string for SQLAlchemy
db=SQLDatabase.from_uri(connection_string)
db

print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM t_order;")

mssql
['t_device', 't_employee', 't_hu_master', 't_order', 't_pick_detail', 't_stored_item']


'[(37339542, \'US1\', 546942427, None, None, None, \'NA34818491_EX\', \'Cindy Poteete\', None, None, None, None, None, 0, None, None, None, \'United Parcel Service\', None, \'PREPAID\', None, 50, \'28:31.0\', \'00:00.0\', \'00:00.0\', \'00:00.0\', \'00:00.0\', \'59:00.0\', 2.71, 0.83, 0, \'N\', None, 0, 0, 0, 0, None, \'Cindy Poteete                 NA34817174\', \'Cindy Poteete                 NA34817174\', \'4702 Harpeth Peytonsville Road\', None, None, "Thompson\'s Station", \'TN\', 37179, \'US\', None, \'6156560985\', \'Cindy Poteete                 NA34817174\', \'Cindy Poteete                 \', \'4702 Harpeth Peytonsville Road\', None, None, "Thompson\'s Station", \'TN\', \'37179\', \'US\', None, \'6156560985\', None, None, None, None, None, None, None, None, None, None, None, \'Cindy Poteete                 \', \'4702 Harpeth Peytonsville Road\', None, None, "Thompson\'s Station", \'TN\', 37179, \'US\', None, \'6156560985\', None, None, None, None, None, None, None, None, None

In [5]:
import re

def clean_sql_query(text: str) -> str:
    """
    Clean SQL query by extracting the SQL code from code blocks or text,
    removing code block syntax, various SQL tags, backticks, square brackets,
    prefixes, and unnecessary whitespace while preserving the core SQL query.

    Args:
        text (str): Raw SQL query text that may contain code blocks, tags, and backticks

    Returns:
        str: Cleaned SQL query
    """
    # Step 1: Extract SQL from code blocks if present
    code_block_pattern = r"```(?:sql|SQL|SQLQuery|mysql|postgresql)?\s*(.*?)\s*```"
    code_blocks = re.findall(code_block_pattern, text, flags=re.DOTALL)
    
    if code_blocks:
        sql = code_blocks[0]
    else:
        # Step 2: Extract SQL statements without requiring semicolons
        sql_statement_pattern = r"(?i)\b(SELECT|INSERT|UPDATE|DELETE)\b.*?(?=;|$)"
        sql_match = re.search(sql_statement_pattern, text, flags=re.DOTALL)
        sql = sql_match.group().strip() if sql_match else ""
    
    # Step 3: Remove non-SQL prefixes (e.g., "AI:") and clean formatting
    sql = re.sub(r'^\s*AI:\s*', '', sql, flags=re.IGNORECASE)  # Remove "AI:" prefix
    sql = re.sub(r'`|\[|\]', '', sql)  # Remove backticks and brackets
    sql = re.sub(r'\s+', ' ', sql).strip()  # Normalize whitespace
    
    return sql

In [6]:
examples = [
    {
        "input": "Give me records of items stuck in carton error",
        "query": "SELECT order_number, status, error_message, t.imported_date, order_type, total_lines, total_units, * FROM t_order t (NOLOCK) WHERE status = 'CRTNERROR' ORDER BY t.imported_date DESC"
    },
    {
        "input": "Orders with no t_pick_detail_cartonize record",
        "query": "SELECT ord.order_number, ord.status, ord.host_status, pdc.status, pdc.order_number, imported_date FROM t_order ord (NOLOCK) LEFT JOIN t_pick_detail_cartonize pdc (NOLOCK) ON ord.order_number = pdc.order_number WHERE imported_date < DATEADD(mi, -5, GETDATE()) AND pdc.order_number IS NULL"
    },
    {
        "input": "Give me records where picked inventory is zero",
        "query": "SELECT * FROM t_stored_item sto (NOLOCK) WHERE actual_qty = 0 AND type <> 0"
    },
    {
        "input": "Give me details for picking status, container ID (LPN) for the order pick line (line_number=1), pull lines (line_number!=1) for the order number or packwaves",
        "query": "SELECT TOP 2 pick_id, order_number, line_number, work_type, status, item_number, planned_quantity, picked_quantity, staged_quantity, pick_location, wave_id, pack_wave_id, load_id, container_id, user_assigned, wh_id, created_date, pick_location_change_date, prev_pick_id, priority, picked_date, * FROM t_pick_detail (NOLOCK) WHERE order_number IN ('')"
    },
    {
        "input": "Give me LPNs details which are of type inventory (IV), loaded (LO), replen (RP), staged order (SO), wave pulls (WP) or LPN orders associated with orders (control_number) or LPNs in status available (A), HOLD (H) or pallet number (parent_hu_id) or label_status in (NULL, COMPLETE, ERROR, PANDA, PREP, PROCESSED)",
        "query": "SELECT * FROM t_hu_master WHERE type IN ('IV', 'LO', 'RP', 'SO', 'WP') OR control_number IS NOT NULL OR status IN ('A', 'H') OR parent_hu_id IS NOT NULL OR label_status IN (NULL, 'COMPLETE', 'ERROR', 'PANDA', 'PREP', 'PROCESSED')"
    },
    {
        "input": "Give me employee details, user is logged into device if device_id is not NULL",
        "query": "SELECT id, name, emp_number, dept, supervisor, menu_level, wh_id, hu_id, device, change_pick_type_allowed, picking_type, can_clear_pallet_positions_flag, chute_packout_full_scan, allow_move_hold_items, login_time, logout_time, last_activity, role, email, phone_number, address, * FROM t_employee WHERE device_id IS NOT NULL"
    },
    {
        "input": "Please provide me details of items stored. If type=0, it's not allocated for any orders. If type is other than (0, -99), it has a foreign reference to t_pick_detail with t_stored_item.type=t_pick_detail.pick",
        "query": "SELECT item_number, actual_qty, unavailable_qty, status, wh_id, location_id, expiration_date, lot_number, inspection_code, serial_number, type, put_away_location, stored_attribute_id, hu_id, shipment_number, item_name, supplier_id, batch_number, created_date, updated_date, * FROM t_stored_item WHERE (type = 0 OR type NOT IN (0, -99)) AND EXISTS (SELECT 1 FROM t_pick_detail WHERE t_stored_item.type = t_pick_detail.pick)"
    },
    {
        "input": "Give me user details, it has device_id (users logged on)",
        "query": "SELECT user_id, user_name, email, phone_number, role, department, supervisor_id, login_time, logout_time, last_activity, device_id, device_type, employee_id, fork_id, wh_id, business_process, text, last_tran_datetime, last_process, last_message_id, last_response_json, * FROM t_user WHERE device_id IS NOT NULL"
    },
    {
        "input": "Give me LPNs which are on hold",
        "query": "SELECT * FROM t_hu_master (NOLOCK) WHERE hu_id IN (SELECT container_number FROM t_pick_detail (NOLOCK) WHERE order_number IN ('')) AND status NOT IN ('A')"
    }
]

In [7]:
from langchain.chains import create_sql_query_chain

from langchain_core.runnables import RunnablePassthrough, RunnableLambda

generate_query=create_sql_query_chain(llm_deepseek, db)

query=generate_query.invoke({"question":"Give me count of all orders"})

print(query)

SELECT COUNT(*) AS order_count FROM t_order


In [8]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
execute_query=QuerySQLDataBaseTool(db=db)
execute_query.invoke(query)

C:\Users\dhruv\AppData\Local\Temp\ipykernel_11664\4290217317.py:2: LangChainDeprecationWarning: The class `QuerySQLDataBaseTool` was deprecated in LangChain 0.3.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-community package and should be used instead. To use it run `pip install -U :class:`~langchain-community` and import as `from :class:`~langchain_community.tools import QuerySQLDatabaseTool``.
  execute_query=QuerySQLDataBaseTool(db=db)


'[(50,)]'

In [9]:
chain = generate_query| RunnableLambda(clean_sql_query) | execute_query

chain.invoke({"question" : "Give me all users name"})

"[('Edward', 'Hernandez'), ('Fernando', 'Navarrete'), ('Melissa', 'Gomez'), ('Francisco', 'Villanueva'), ('Douglas', 'Johnson'), ('Joseph', 'Sepulveda'), ('August', 'Torres'), ('Sergio', 'Aquino'), ('Vitech', 'Sustainment'), ('Patricio', 'Ponce'), ('Alvaro', 'Alvarado'), ('Richard', 'Lopez'), ('Alba', 'Escandon'), ('Adrian', 'Bacerra'), ('Isabel', 'Magana'), ('Mario', 'Duran'), ('Willie', 'Garcia'), ('Ty', 'Ty'), ('Jonathan', 'Arteaga'), ('Gibram', 'Macias'), ('Sara', 'Navarro'), ('Maria', 'Prado'), ('Jerson', 'Salazar'), ('Citlali', 'Borja'), ('Erica', 'Garcia'), ('Jean', 'Arango'), ('Jasmine', 'Silva'), ('CS', 'CS'), ('Angel', 'Rmirez'), ('Alejandra', 'Hernandez'), ('Christopher', 'Bulter'), ('Joshua', 'Ritchie'), ('Edward', 'Garcia'), ('Laura', 'Solis'), ('Rashelle', 'Vasquez'), ('Jessica', 'Martinez'), ('Mireya', 'Maragadao'), ('Oscar', 'Serratos'), ('Javier', 'Ornelas'), ('Victor', 'Miranda'), ('Wendy', 'Mendoza'), ('David', 'Asencio'), ('Veronica', 'Martinez'), ('Cory', 'Loffer')

In [20]:
chain.get_prompts()[0].pretty_print()

You are an MS SQL expert. Given an input question, first create a syntactically correct MS SQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the TOP clause as per MS SQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in square brackets ([]) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CAST(GETDATE() as date) function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQL

In [10]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt=PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query and SQL result, answer the user question.
    Question:{question}
    SQL Query :{query}
    SQL Result : {result}
    Answer : 
"""
)


rephrase_answer =answer_prompt | llm_deepseek | StrOutputParser()

chain= (
    RunnablePassthrough.assign(query=generate_query |RunnableLambda(clean_sql_query)).assign(
        result=itemgetter("query") | execute_query
    )
    |
    rephrase_answer
)

result=chain.invoke({"question":"Give me count of all orders"})

In [11]:
print(result)

</think>

The count of all orders is **50**.


****Few Shot Prompt Example****

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, FewShotChatMessagePromptTemplate, PromptTemplate

example_prompt= ChatPromptTemplate.from_messages(
    [
        ("human","{input}\nSQLQuery:"),
        ("ai","{query}"),
    ]
)

few_shot_prompt=FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    input_variables=["input"],
    
)

print(few_shot_prompt.format(input="How many orders are there"))

Human: Give me records of items stuck in carton error
SQLQuery:
AI: SELECT order_number, status, error_message, t.imported_date, order_type, total_lines, total_units, * FROM t_order t (NOLOCK) WHERE status = 'CRTNERROR' ORDER BY t.imported_date DESC
Human: Orders with no t_pick_detail_cartonize record
SQLQuery:
AI: SELECT ord.order_number, ord.status, ord.host_status, pdc.status, pdc.order_number, imported_date FROM t_order ord (NOLOCK) LEFT JOIN t_pick_detail_cartonize pdc (NOLOCK) ON ord.order_number = pdc.order_number WHERE imported_date < DATEADD(mi, -5, GETDATE()) AND pdc.order_number IS NULL
Human: Give me records where picked inventory is zero
SQLQuery:
AI: SELECT * FROM t_stored_item sto (NOLOCK) WHERE actual_qty = 0 AND type <> 0
Human: Give me details for picking status, container ID (LPN) for the order pick line (line_number=1), pull lines (line_number!=1) for the order number or packwaves
SQLQuery:
AI: SELECT TOP 2 pick_id, order_number, line_number, work_type, status, item

***Dynamic Few Shot Example Selector***


In [13]:
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_huggingface import HuggingFaceEndpointEmbeddings 
from langchain_huggingface import HuggingFaceEmbeddings

model_name="sentence-transformers/all-MiniLM-L6-v2"

hf=HuggingFaceEndpointEmbeddings(
    model=model_name
)

vectorstore= Chroma()
vectorstore.delete_collection()
example_selector=SemanticSimilarityExampleSelector.from_examples(
    examples,
    HuggingFaceEndpointEmbeddings(),
    vectorstore,
    k=3,
    input_keys=["input"],  
)

example_selector.select_examples({"input":"Give me LPN details"})

C:\Users\dhruv\AppData\Local\Temp\ipykernel_11664\2633461519.py:12: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore= Chroma()


[{'input': 'Give me LPNs which are on hold',
  'query': "SELECT * FROM t_hu_master (NOLOCK) WHERE hu_id IN (SELECT container_number FROM t_pick_detail (NOLOCK) WHERE order_number IN ('')) AND status NOT IN ('A')"},
 {'input': 'Give me LPNs details which are of type inventory (IV), loaded (LO), replen (RP), staged order (SO), wave pulls (WP) or LPN orders associated with orders (control_number) or LPNs in status available (A), HOLD (H) or pallet number (parent_hu_id) or label_status in (NULL, COMPLETE, ERROR, PANDA, PREP, PROCESSED)',
  'query': "SELECT * FROM t_hu_master WHERE type IN ('IV', 'LO', 'RP', 'SO', 'WP') OR control_number IS NOT NULL OR status IN ('A', 'H') OR parent_hu_id IS NOT NULL OR label_status IN (NULL, 'COMPLETE', 'ERROR', 'PANDA', 'PREP', 'PROCESSED')"},
 {'input': 'Give me employee details, user is logged into device if device_id is not NULL',
  'query': 'SELECT id, name, emp_number, dept, supervisor, menu_level, wh_id, hu_id, device, change_pick_type_allowed, pick

***Customising Few Shot Prompt Template***


In [14]:
few_shot_prompt=FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    input_variables=["input","top_k"],
    
)

print(few_shot_prompt.format(input="How many orders are there",top_k=3, table_info=""))

Human: Give me details for picking status, container ID (LPN) for the order pick line (line_number=1), pull lines (line_number!=1) for the order number or packwaves
SQLQuery:
AI: SELECT TOP 2 pick_id, order_number, line_number, work_type, status, item_number, planned_quantity, picked_quantity, staged_quantity, pick_location, wave_id, pack_wave_id, load_id, container_id, user_assigned, wh_id, created_date, pick_location_change_date, prev_pick_id, priority, picked_date, * FROM t_pick_detail (NOLOCK) WHERE order_number IN ('')
Human: Give me LPNs details which are of type inventory (IV), loaded (LO), replen (RP), staged order (SO), wave pulls (WP) or LPN orders associated with orders (control_number) or LPNs in status available (A), HOLD (H) or pallet number (parent_hu_id) or label_status in (NULL, COMPLETE, ERROR, PANDA, PREP, PROCESSED)
SQLQuery:
AI: SELECT * FROM t_hu_master WHERE type IN ('IV', 'LO', 'RP', 'SO', 'WP') OR control_number IS NOT NULL OR status IN ('A', 'H') OR parent_hu_

In [15]:
final_prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a MS SQL Expert. Given an imput question, create a syntactically correct MS SQL query to run,and return the answer to the input question.Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in square brackets ([]) to denote them as delimited identifiers.Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.Pay attention to use CAST(GETDATE() as date) function to get the current date, if the question involves ""today"". Do not add any extra chracter which will impact the sql query excution. \n\n Here is the relevant table info:{table_info}\n\n Below are number of examples of question and their corresponding MSSQL queries.Please output the plain sql query"),
        few_shot_prompt,
        ("human","{input}"),
    ]
)

print(final_prompt.format(input="How many orders are there", table_info="show some table info"))

System: You are a MS SQL Expert. Given an imput question, create a syntactically correct MS SQL query to run,and return the answer to the input question.Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in square brackets ([]) to denote them as delimited identifiers.Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.Pay attention to use CAST(GETDATE() as date) function to get the current date, if the question involves today. Do not add any extra chracter which will impact the sql query excution. 

 Here is the relevant table info:show some table info

 Below are number of examples of question and their corresponding MSSQL queries.Please output the plain sql query
Human: Give me details for picking status, container ID (LPN) for the order pick line (line_number=1), pull 

In [22]:
generate_query=create_sql_query_chain(llm_openai,db,final_prompt)


chain= (
    RunnablePassthrough.assign(query=generate_query| RunnableLambda(clean_sql_query)).assign(
        result=itemgetter("query") | execute_query
    )
    |
    rephrase_answer
)

result=chain.invoke({"question":"Give me all employees names"})





In [23]:

print(result)


</think>

Here is a list of all employee names:

1. Edward Hernandez
2. Fernando Navarrete
3. Melissa Gomez
4. Francisco Villanueva
5. Douglas Johnson
6. Joseph Sepulveda
7. August Torres
8. Sergio Aquino
9. Vitech Sustainment
10. Patricio Ponce
11. Alvaro Alvarado
12. Richard Lopez
13. Alba Escandon
14. Adrian Bacerra
15. Isabel Magana
16. Mario Duran
17. Willie Garcia
18. Ty Ty
19. Jonathan Arteaga
20. Gibram Macias
21. Sara Navarro
22. Maria Prado
23. Jerson Salazar
24. Citlali Borja
25. Erica Garcia
26. Jean Arango
27. Jasmine Silva
28. CS CS
29. Angel Rmirez
30. Alejandra Hernandez
31. Christopher Bulter
32. Joshua Ritchie
33. Edward Garcia
34. Laura Solis
35. Rashelle Vasquez
36. Jessica Martinez
37. Mireya Maragadao
38. Oscar Serratos
39. Javier Ornelas
40. Victor Miranda
41. Wendy Mendoza
42. David Asencio
43. Veronica Martinez
44. Cory Loffer
45. Lucia Melgar
46. Ashley Molina
47. Miranda Ochoa
48. Crystal Noa
49. Ivan Espita
50. Kenneth Ferrell
51. Michael Vera
52. Gustavo Hu

In [132]:

print(result)

</think>

Here is a list of all employee names based on the provided SQL result:

1. Edward Hernandez
2. Fernando Ararrete
3. Melissa Gomez
4. Francisco Villanueva
5. Douglas Johnson
6. Joseph Sepulveda
7. August Torres
8. Sergio Aquino
9. Vitech Sustainment
10. Patricio Ponce
11. Alvaro Alvarado
12. Richard Lopez
13. Alba Escandon
14. Adrian Bacerra
15. Isabel Magana
16. Mario Duran
17. Willie Garcia
18. Ty Ty
19. Jonathan Arteaga
20. Gibram Macias
21. Sara Navarro
22. Maria Prado
23. Jerson Salazar
24. Citlali B'Navarrete
25. Erica Garcia
26. Jean Arango
27. Jasmine Silva
28. CS CS
29. Angel Rmirez
30. Alejandra Hernandez
31. Christopher Bulter
32. Joshua Ritchie
33. Edward Garcia
34. Laura Solis
35. Rashelle Vasquez
36. Jessica Martinez
37. Mireya Maragadao
38. Oscar Serratos
39. Javier Ornelas
40. Victor Miranda
41. Wendy Mendoza
42. David Asencio
43. Veronica Martinez
44. Cory Loffer
45. Lucia Melgar
46. Ashley Molina
47. Miranda Ochoa
48. Crystal Noa
49. Ivan Espita
50. Kenneth F

**Dynamic Relevant Table Selection**

In [24]:
from operator import itemgetter
from langchain.chains.openai_tools import create_extraction_chain_pydantic
from pydantic import BaseModel, Field
from typing import List
import pandas as pd

def get_table_details():
    ##Read CSV file into dataframe
    table_description=pd.read_csv("tables_description.csv")
    table_docs=[]


    #Iterate over dataframe rows to create Document objects
    table_details=""
    for index, row in table_description.iterrows():
        table_details=table_details + "Table Name:" + row['Table'] + "\n" + "Table Description:" + row['Description'] + "\n\n"

    return table_details

class Table(BaseModel):

    """Table in SQL Database"""

    name: str = Field(description="Name of all table in SQL databases.")

table_details=get_table_details()
print(table_details)    


Table Name:t_employee
Table Description:Stores information about employees, including the equipment(device) the employee has logged on to and the menu level to display to the employee.

Table Name:t_device
Table Description:Stores the information related to device, employee_id logged in, fork_id, last transaction time, business process, device type etc.

Table Name:t_hu_master
Table Description:This table lists handling units (license plates LPNs) of related items that are stored, moved, and/or picked together.
It contains type The type of current license plate record. Valid values are: AR – Anticipated Receipt IV – Inventory LO – Loaded Order RC – Received SO – Staged Order .    Control number. Types IV and AR may have an ASN, receiver, or receipt identifier as a control number. Types LO and SO may have an order number or work order identifier as a control number, Valid location identifier in the warehouse, status-Identifies whether a license plate is in use or not, as well as indicat

In [25]:
table_details_prompt=f"""Return the name of all the SQL Tables that might be relevant to the user question.\n
The tables are:
{table_details}
Remember to include all potentially Relevant table, even if you're not sure that they are needed. """

table_chain = create_extraction_chain_pydantic(Table,llm_groq,system_message=table_details_prompt)
tables=table_chain.invoke({"input":"Give me count of  orders shipped to each state"})
tables


C:\Users\dhruv\AppData\Local\Temp\ipykernel_11664\2623611836.py:6: LangChainDeprecationWarning: LangChain has introduced a method called `with_structured_output` thatis available on ChatModels capable of tool calling.You can read more about the method here: <https://python.langchain.com/docs/modules/model_io/chat/structured_output/>. Please follow our extraction use case documentation for more guidelineson how to do information extraction with LLMs.<https://python.langchain.com/docs/use_cases/extraction/>. with_structured_output does not currently support a list of pydantic schemas. If this is a blocker or if you notice other issues, please provide feedback here:<https://github.com/langchain-ai/langchain/discussions/18154>
  table_chain = create_extraction_chain_pydantic(Table,llm_groq,system_message=table_details_prompt)


[Table(name='t_order')]

In [26]:
def get_tables(tables: List[Table]) -> List[str]:
    tables=[table.name for table in tables]
    return tables

select_tables={"input":itemgetter("question")}|create_extraction_chain_pydantic(Table, llm_openai, system_message=table_details_prompt)| get_tables
select_tables.invoke({"question": "Give me count of orders shipped to each state"}) 

['t_order']

In [27]:
generate_query=create_sql_query_chain(llm_openai,db,final_prompt)

chain=(
    RunnablePassthrough.assign(table_name_to_use=select_tables) |
    RunnablePassthrough.assign(query=generate_query | RunnableLambda(clean_sql_query)).assign(
        result=itemgetter("query") | execute_query
    )
    |
    rephrase_answer
)

result=chain.invoke({"question": "Give me count of orders shipped to each state"})
print(result)

</think>

Here is the count of orders shipped to each state:

- AL: 1
- BC: 1
- CA: 5
- CO: 2
- CT: 1
- FL: 1
- GA: 2
- IA: 1
- IL: 3
- MA: 2
- MD: 3
- MI: 1
- MO: 1
- NC: 1
- NH: 1
- NJ: 2
- NY: 9
- OH: 2
- OK: 1
- TN: 1
- TX: 4
- VA: 2
- WA: 1
- WI: 2


In [28]:
table_details_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", """Return the names of ALL the SQL tables that MIGHT be relevant to the user question.
                          The tables are:

                          {table_details}

                          Remember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed."""),
            ("human", "{question}")
        ]
    )

structured_llm = llm_openai.with_structured_output(Table)

table_chain = table_details_prompt | structured_llm
tables = table_chain.invoke({"question": "Give me count of orders shipped to each state", "table_details":table_details})
tables
     

Table(name='t_order')

In [29]:
def get_tables(table_response: Table) -> List[str]:
    """
    Extracts the list of table names from a Table object.

    Args:
        table_response (Table): A Pydantic Table object containing table names.

    Returns:
        List[str]: A list of table names.
    """
    if isinstance(table_response.name, str):
        return [table_response.name]
    elif isinstance(table_response.name, list):
        return table_response.name
    else:
        # Handle unexpected types
        return []

select_table = {"question": itemgetter("question"), "table_details": itemgetter("table_details")} | table_chain | get_tables
select_table.invoke({"question": "give me details of customer and their order count", "table_details":table_details})

['t_order']

In [30]:
generate_query=create_sql_query_chain(llm_openai,db,final_prompt)
chain = (
RunnablePassthrough.assign(table_names_to_use=select_table) |
RunnablePassthrough.assign(query=generate_query| RunnableLambda(clean_sql_query)).assign(
    result=itemgetter("query") | execute_query
)
| rephrase_answer
)


In [31]:
result=chain.invoke({"question": "give me count of orders shipped to each state", "table_details":table_details})
print(result)

</think>

The count of orders shipped to each state is as follows:

- AL: 1
- CA: 7
- CO: 2
- FL: 1
- GA: 2
- IA: 1
- IL: 3
- MA: 2
- MD: 3
- MI: 1
- MO: 1
- NC: 1
- NH: 1
- NJ: 2
- NY: 9
- OH: 2
- OK: 1
- TN: 1
- TX: 3
- VA: 2
- VER: 1
- WA: 1
- WI: 2


In [32]:
result=chain.invoke({"question": "Can you give me details about the order_number 546942489 ", "table_details":table_details})
print(result)

</think>

The order with the number 546942489 has the following details:

- **Order ID:** 37339496
- **Warehouse ID:** US1
- **Customer Name:** Melanie Chacha
- **Status:** CARTNERROR
- **Order Date:** 23:55.0
- **Arrive Date:** 00:00.0
- **Weight:** 4.49
- **Freight Cost:** 0


In [33]:
print(result)

</think>

The order with the number 546942489 has the following details:

- **Order ID:** 37339496
- **Warehouse ID:** US1
- **Customer Name:** Melanie Chacha
- **Status:** CARTNERROR
- **Order Date:** 23:55.0
- **Arrive Date:** 00:00.0
- **Weight:** 4.49
- **Freight Cost:** 0


In [34]:
print(result)

</think>

The order with the number 546942489 has the following details:

- **Order ID:** 37339496
- **Warehouse ID:** US1
- **Customer Name:** Melanie Chacha
- **Status:** CARTNERROR
- **Order Date:** 23:55.0
- **Arrive Date:** 00:00.0
- **Weight:** 4.49
- **Freight Cost:** 0


Adding Memory to Chatbot so that it answers follow-up questions related to the database

In [35]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries. Those examples are just for referecne and hsould be considered while answering follow up questions"),
        few_shot_prompt,
        MessagesPlaceholder(variable_name="messages"),
        ("human", "{input}"),
    ]
)
print(final_prompt.format(input="How many items are there?",table_info="some table info",messages=[]))

System: You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.

Here is the relevant table info: some table info

Below are a number of examples of questions and their corresponding SQL queries. Those examples are just for referecne and hsould be considered while answering follow up questions
Human: Give me records where picked inventory is zero
SQLQuery:
AI: SELECT * FROM t_stored_item sto (NOLOCK) WHERE actual_qty = 0 AND type <> 0
Human: Give me records of items stuck in carton error
SQLQuery:
AI: SELECT order_number, status, error_message, t.imported_date, order_type, total_lines, total_units, * FROM t_order t (NOLOCK) WHERE status = 'CRTNERROR' ORDER BY t.imported_date DESC
Human: Please provide me details of items stored. If type=0, it's not allocated for any orders. If type is other than (0, -99), it has a foreign reference to t_pick_detail with t_stored_item.type=t_pick_detail.pick
SQLQuery:
AI: SELECT it

In [36]:
from langchain.memory import ChatMessageHistory
history = ChatMessageHistory()

generate_query = create_sql_query_chain(llm_deepseek, db,final_prompt)

chain = (
RunnablePassthrough.assign(table_names_to_use=select_table) |
RunnablePassthrough.assign(query=generate_query | RunnableLambda(clean_sql_query)).assign(
    result=itemgetter("query") | execute_query
)
| rephrase_answer
)

In [37]:

question = "What is the shipping address for order_number 546942489"
response = chain.invoke({"question": question,"messages":history.messages, "table_details":table_details})
response

'</think>\n\nThe shipping address for order_number 546942489 is as follows:\n\n- **Name:** Melanie Chacha  \n- **Address Line 1:** 35 Suydam St  \n- **Address Line 2:** Apt 2  \n- **City:** Brooklyn  \n- **State:** NY  \n- **Zip Code:** 11221  \n- **Country Code:** US'

In [38]:
print(response)

</think>

The shipping address for order_number 546942489 is as follows:

- **Name:** Melanie Chacha  
- **Address Line 1:** 35 Suydam St  
- **Address Line 2:** Apt 2  
- **City:** Brooklyn  
- **State:** NY  
- **Zip Code:** 11221  
- **Country Code:** US


In [39]:
history.add_user_message(question)
history.add_ai_message(response)
history.messages
     


[HumanMessage(content='What is the shipping address for order_number 546942489', additional_kwargs={}, response_metadata={}),
 AIMessage(content='</think>\n\nThe shipping address for order_number 546942489 is as follows:\n\n- **Name:** Melanie Chacha  \n- **Address Line 1:** 35 Suydam St  \n- **Address Line 2:** Apt 2  \n- **City:** Brooklyn  \n- **State:** NY  \n- **Zip Code:** 11221  \n- **Country Code:** US', additional_kwargs={}, response_metadata={})]

In [40]:

response = chain.invoke({"question": "Can you status and carrier code of of that order?","messages":history.messages, "table_details":table_details})
response
     

'</think>\n\nThe status of the order is **CARTNERROR** and the carrier code is **000001_UPS_P_RES**.'

In [260]:
print(response)

</think>

The status of the order is **CARTNERROR** and the carrier code is **000001_UPS_P_RES**.
